In [1]:
from anthropic import AnthropicBedrock
from application import Application
from services import CompilerService
import tempfile

In [2]:
compiler = CompilerService()
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [3]:

tempdir = tempfile.TemporaryDirectory()
application = Application(client, compiler, "templates", tempdir.name)

In [4]:
application_description = """
Bot that tracks my exercise routine in the gym, tracks progress and suggests new routines
for specific list of available equipment and time constraints.
""".strip()

In [5]:
my_bot = application.create_bot(application_description)

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


Compiling TypeSpec...
Generating Typescript Schema Definitions...
Compiling Drizzle...
Generating Router...
Generating Preprocessors...
Generating Handlers...
Generating Application...


In [6]:
my_bot

{'typespec': {'message': {'role': 'assistant',
   'content': '<reasoning>\nFor a gym tracking bot, I expect users to send messages like:\n"I did 3 sets of bench press with 80kg" \n"Show my progress for bench press"\n"Suggest a 30-minute chest workout with dumbbells and bench"\n\nKey components needed:\n1. Exercise records with sets, reps, and weights\n2. Equipment list for availability constraints\n3. Workout routine suggestions based on time and equipment\n4. Progress tracking over time\n5. Exercise categorization by muscle groups\n\nLLM can extract:\n- Exercise details from natural descriptions\n- Time constraints from phrases like "30-minute workout"\n- Equipment mentions from the message\n- Muscle group targets from context\n</reasoning>\n\n<typespec>\nmodel Exercise {\n  name: string;\n  muscleGroup: string;\n  equipment: string[];\n  sets: Set[];\n}\n\nmodel Set {\n  reps: integer;\n  weight: float;\n  restTime: duration;\n}\n\nmodel WorkoutRoutine {\n  exercises: Exercise[];\n  

In [7]:
application.generation_dir



'/tmp/tmp1kp0xrww/generated/generation-1'

In [ ]:
!pushd {application.generation_dir}/app_schema
!npm install
!npx tsc
!popd


npm WARN deprecated @esbuild-kit/esm-loader@2.6.5: Merged into tsx: https://tsx.isK
npm WARN deprecated @esbuild-kit/core-utils@3.3.2: Merged into tsx: https://tsx.is
m##################) ⠴ reify:drizzle-orm: timing reifyNode:node_modules/typescr/@smits/@
added 404 packages, and audited 405 packages in 8s

6 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities
src/handlers/get_progress.ts:39:27 - error TS2339: Property 'createdAt' does not exist on type 'PgTableWithColumns<{ name: "sets"; schema: undefined; columns: { id: PgColumn<{ name: "id"; tableName: "sets"; dataType: "number"; columnType: "PgInteger"; data: number; driverParam: string | number; notNull: true; hasDefault: true; ... 6 more ...; generated: undefined; }, {}, {}>; exerciseId: PgColumn<...>; reps: P...'.

39         between(setsTable.createdAt, options.from, options.to)
                             ~~~~~~~~~

src/handlers/get_progress.ts:42:24 - error TS2339: Property 'createdAt' does 